In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer
from datetime import datetime

import lightgbm as gbm

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample = pd.read_csv('data/sample.csv',index_col=0,header=None)

In [3]:
train

,datetime,y,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature
0,2013-11-18,90,月,0,厚切りイカフライ,NaN,NaN,NaN,NaN,快晴,--,19.8
1,2013-11-19,101,火,1,手作りヒレカツ,NaN,NaN,NaN,NaN,快晴,--,17.0
2,2013-11-20,118,水,0,白身魚唐揚げ野菜あん,NaN,NaN,NaN,NaN,快晴,--,15.5
3,2013-11-21,120,木,1,若鶏ピリ辛焼,NaN,NaN,NaN,NaN,快晴,--,15.2
4,2013-11-22,130,金,1,ビッグメンチカツ,NaN,NaN,NaN,NaN,快晴,--,16.1
...,...,...,...,...,...,...,...,...,...,...,...,...
202,2014-9-24,59,水,1,白身魚のマスタード焼き,408.0,NaN,NaN,NaN,曇,0,24.8
203,2014-9-25,50,木,0,牛カルビ焼き肉,394.0,NaN,NaN,NaN,曇,0,25.4
204,2014-9-26,45,金,0,ランチビュッフェ,NaN,スペシャルメニュー（800円）,キャリアアップ支援セミナー,NaN,晴れ,--,27.1
205,2014-9-29,56,月,1,豚肉と玉子の炒め,404.0,NaN,NaN,NaN,快晴,--,26.6


In [4]:
#データ確認・前処理などなど
train_index = pd.to_datetime(train['datetime'])
test_index = pd.to_datetime(test['datetime'])

In [5]:
train.describe()

,y,soldout,kcal,payday,temperature
count,207.000000,207.000000,166.000000,10.0,207.000000
mean,86.623188,0.449275,404.409639,1.0,19.252174
std,32.882448,0.498626,29.884641,0.0,8.611365
min,29.000000,0.000000,315.000000,1.0,1.200000
25%,57.000000,0.000000,386.000000,1.0,11.550000
50%,78.000000,0.000000,408.500000,1.0,19.800000
75%,113.000000,1.000000,426.000000,1.0,26.100000
max,171.000000,1.000000,462.000000,1.0,34.600000


In [6]:
#降水量の格納データ
train.precipitation.unique()

array(['--', '0.5', '0', '1.5', '1', '6', '6.5', '2.5'], dtype=object)

In [7]:
train.describe(include='O')

,datetime,week,name,remarks,event,weather,precipitation
count,207,207,207,21,14,207,207
unique,207,5,156,6,2,7,8
top,2013-11-18,水,メンチカツ,お楽しみメニュー,ママの会,快晴,--
freq,1,43,6,12,9,53,169


In [19]:
ohe_columns = ['week','name','remarks','event','weather','precipitation']

In [36]:
train_y = train.loc[:,'y']
train_x = train.drop('y' , axis=1)

In [37]:
data_all = pd.concat([train_x , test])
data_all

,datetime,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature
0,2013-11-18,月,0,厚切りイカフライ,NaN,NaN,NaN,NaN,快晴,--,19.8
1,2013-11-19,火,1,手作りヒレカツ,NaN,NaN,NaN,NaN,快晴,--,17.0
2,2013-11-20,水,0,白身魚唐揚げ野菜あん,NaN,NaN,NaN,NaN,快晴,--,15.5
3,2013-11-21,木,1,若鶏ピリ辛焼,NaN,NaN,NaN,NaN,快晴,--,15.2
4,2013-11-22,金,1,ビッグメンチカツ,NaN,NaN,NaN,NaN,快晴,--,16.1
5,2013-11-25,月,1,鶏の唐揚,NaN,NaN,NaN,NaN,曇,--,14.6
6,2013-11-26,火,0,豚のスタミナ炒め,NaN,NaN,NaN,NaN,快晴,--,17.9
7,2013-11-27,水,1,ボローニャ風カツ,NaN,NaN,NaN,NaN,晴れ,--,14.7
8,2013-11-28,木,0,ハンバーグ,NaN,NaN,NaN,NaN,薄曇,--,17.7
9,2013-11-29,金,0,タルタルinソーセージカツ,NaN,NaN,NaN,NaN,快晴,--,12.1


In [38]:
pd.set_option('display.max_rows' , 500)
data_all_ohe = pd.get_dummies(data_all , dummy_na=True , columns=ohe_columns)
data_all_ohe

,datetime,soldout,kcal,payday,temperature,week_月,week_木,week_水,week_火,week_金,...,weather_nan,precipitation_--,precipitation_0,precipitation_0.5,precipitation_1,precipitation_1.5,precipitation_2.5,precipitation_6,precipitation_6.5,precipitation_nan
0,2013-11-18,0,NaN,NaN,19.8,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,2013-11-19,1,NaN,NaN,17.0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
2,2013-11-20,0,NaN,NaN,15.5,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,2013-11-21,1,NaN,NaN,15.2,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,2013-11-22,1,NaN,NaN,16.1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
5,2013-11-25,1,NaN,NaN,14.6,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6,2013-11-26,0,NaN,NaN,17.9,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
7,2013-11-27,1,NaN,NaN,14.7,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
8,2013-11-28,0,NaN,NaN,17.7,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
9,2013-11-29,0,NaN,NaN,12.1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0


In [42]:
#datetimeで特徴量を作成
#yearカラムはデータ数が少ないため除外
data_all_ohe['datetime'] = pd.to_datetime(data_all_ohe['datetime'])
data_all_ohe

,datetime,soldout,kcal,payday,temperature,week_月,week_木,week_水,week_火,week_金,...,weather_nan,precipitation_--,precipitation_0,precipitation_0.5,precipitation_1,precipitation_1.5,precipitation_2.5,precipitation_6,precipitation_6.5,precipitation_nan
0,2013-11-18,0,NaN,NaN,19.8,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,2013-11-19,1,NaN,NaN,17.0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
2,2013-11-20,0,NaN,NaN,15.5,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,2013-11-21,1,NaN,NaN,15.2,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,2013-11-22,1,NaN,NaN,16.1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
5,2013-11-25,1,NaN,NaN,14.6,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6,2013-11-26,0,NaN,NaN,17.9,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
7,2013-11-27,1,NaN,NaN,14.7,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
8,2013-11-28,0,NaN,NaN,17.7,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
9,2013-11-29,0,NaN,NaN,12.1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0


In [43]:
data_all_ohe['month'] = data_all_ohe['datetime'].datetime.month

AttributeError: 'Series' object has no attribute 'datetime'

In [18]:
#欠損値の平均値補完　しようと思ったけど　そのままやります

In [ ]:
#時系列から特徴量を作成


In [ ]:
#時系列データのクロスバリデーション
#5 FoldでCross validation